In [ ]:
# Data Preparation
import json

labeled_data = []
spacy_data = []

with open(r"AllenNLP/admin.jsonl", "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)
        if "label" in line:
            data["entities"] = data.pop("label")
        else:
            data["entities"] = []
            
        tmp_ents = []
        for e in data["entities"]:
#             if e[2] in ['APTFamily', 'ExeFile', 'ScriptsFile']
            tmp_ents.append({"start": e[0], "end": e[1], "label": e[2]})
        
            data["entities"] = tmp_ents
        
        if len(data["data"]) > 5:
            spacy_line = json.dumps({"entities": data["entities"], "text": data["data"]})
            spacy_data.append(spacy_line)
            print(spacy_line)

In [3]:
labeled_data

[{'id': 41208,
  'data': 'Cobalt Strike can use known credentials to run commands and spawn processes as a domain user account.',
  'label': [[0, 13, 'APTFamily'],
   [47, 56, 'ScriptsFile'],
   [66, 76, 'ExeFile']]},
 {'id': 41220,
  'data': 'Tropic Trooper has used known administrator account credentials to execute the backdoor directly.',
  'label': [[0, 14, 'APTFamily'], [79, 88, 'ExeFile']]},
 {'id': 41226,
  'data': 'Bonadan can create bind and reverse shells on the infected system.',
  'label': [[0, 8, 'APTFamily'], [28, 42, 'C2C']]},
 {'id': 41232,
  'data': 'FIN6 has used scripting to iterate through a list of compromised PoS systems, copy data to a log file, and remove the original data files.',
  'label': [[0, 5, 'APTFamily'],
   [14, 24, 'ExeFile'],
   [83, 87, 'DocumentFile'],
   [93, 101, 'DocumentFile'],
   [118, 137, 'DocumentFile']]},
 {'id': 41238,
  'data': 'Kessel can create a reverse shell between the infected host and a specified system.',
  'label': [[0, 7, 'APTF

In [ ]:
import spacy
import random
import json

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
ner.add_label("NetLoc")
ner.add_label("APTFamily")
ner.add_label("ExeFile")
ner.add_label("ScriptsFile")
ner.add_label("DocumentFile")
ner.add_label("E-mail")
ner.add_label("Registry")
ner.add_label("File")
ner.add_label("Vulnerability")
ner.add_label("C2C")
ner.add_label("SensInfo")
ner.add_label("Service")

nlp.begin_training()